In [1]:
from lib import *

In [22]:
@level_ab(1, test=('1000\n2000\n3000\n\n4000\n\n5000\n6000\n\n7000\n8000\n9000\n\n10000', 24000, 45000), sep="\n\n")
def solve(num: List[str], level=0):
    chunks = [sum(map(int, line.split("\n"))) for line in num]
    return sum(list(sorted(chunks, reverse=True))[:3]) if level else max(chunks)

Part a already solved with same answer: 69912
Part b already solved with same answer: 208180


In [23]:
@level_ab(2, test=("""A Y\nB X\nC Z""", 15, 12), sep="\n")
def solve(lines: List[str], level=0):
    total = 0
    for line in lines:
        a, b = line.split(" ")
        a, b = ord(a) - ord('A'), ord(b) - ord('X')
        total += b * 3 + (a+b-1) % 3 + 1 if level else ((b-a+1)%3) * 3 + b + 1
    return total

Part a already solved with same answer: 11603
Part b already solved with same answer: 12725


In [28]:
def prio(c: str) -> int:
    return ord(c.lower()) - ord('a') + 1 + c.isupper() * 26

@level_ab(3, test=("""vJrwpWtwJgWrhcsFMMfFFhFp\njqHRNqRjqzjGDLGLrsFMfFZSrLrFZsSL\nPmmdzqPrVvPwwTWBwg\nwMqvLMZHhHMvwLHjbvcjnnSBnvTQFn\nttgJtRGJQctTZtZT\nCrZsJsPPZsGzwwsLwLmpwMDw""", 157, 70), sep="\n")
def solve(lines: List[str], level=0):
    total = 0
    if level:
        for i in range(0, len(lines)-1, 3):
            a, b, c = map(set, lines[i: i+3])
            total += prio(next(iter(a & b & c)))
    else:
        for line in lines:
            offset = len(line)//2
            a, b = set(line[:offset]), set(line[offset:])
            total += sum([prio(c) for c in a & b])
    return total

Part a already solved with same answer: 8176
Part b already solved with same answer: 2689


In [ ]:
@level_ab(3, test=("""vJrwpWtwJgWrhcsFMMfFFhFp\njqHRNqRjqzjGDLGLrsFMfFZSrLrFZsSL\nPmmdzqPrVvPwwTWBwg\nwMqvLMZHhHMvwLHjbvcjnnSBnvTQFn\nttgJtRGJQctTZtZT\nCrZsJsPPZsGzwwsLwLmpwMDw""", 157, 70), sep="\n")
def solve(lines: List[str], level=0):
    total = 0
    if level:
        for i in range(0, len(lines)-1, 3):
            a, b, c = map(set, lines[i: i+3])
            total += prio(next(iter(a & b & c)))
    else:
        for line in lines:
            offset = len(line)//2
            a, b = set(line[:offset]), set(line[offset:])
            total += sum([prio(c) for c in a & b])
    return total